## 판매량 데이터를 통해 판매금액 데이터 보충

### 2023-02-23 부터 2023-03-28 까지 sales의 총 판매금액이 이상함.

In [1]:
#
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# read_csv
raw_sales = pd.read_csv('sales_Prophet.csv')   # 총 판매금액
raw_train = pd.read_csv('train_Prophet.csv')   # 총 판매량

In [3]:
#
sales = raw_sales.drop(['ID','제품','대분류','중분류','소분류','브랜드'], axis=1).transpose()
train = raw_train.drop(['ID','제품','대분류','중분류','소분류','브랜드'], axis=1).transpose()

In [4]:
#
sales_df = sales.loc[:'2023-02-22',:]
train_df = train.loc[:'2023-02-22',:]

## 1.2022-01-01 부터 2023-02-22까지 데이터를 통해 제품 별 평균 가격을 구함

In [5]:
#
price = pd.DataFrame(index = sales_df.index, columns = sales_df.columns)
#price

In [6]:
#
for j in range(15890):
    for i in range(418):
        if sales_df.iloc[i,j] != 0:
            price.iloc[i,j] = sales_df.iloc[i,j]/train_df.iloc[i,j]
        else:
            price.iloc[i,j] = 0

In [7]:
#
price

,0,1,2,3,4,5,6,7,8,9,...,15880,15881,15882,15883,15884,15885,15886,15887,15888,15889
2022-01-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-04,0,0,0,0,0,0,7000.0,0,4200.0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-05,0,0,0,0,0,0,7045.238095,0,4200.0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-18,0,0,0,0,6450.0,0,0,0,8900.0,14766.666667,...,0,0,0,0,0,0,22712.601626,0,0,0
2023-02-19,0,0,0,0,0,0,0,0,8900.0,0,...,0,0,0,0,0,2000.0,22617.037037,0,0,0
2023-02-20,0,0,0,0,6450.0,0,0,0,8900.0,0,...,0,0,0,0,0,2000.0,22418.421053,0,0,0
2023-02-21,0,0,0,0,6450.0,0,0,0,0,0,...,0,0,0,0,0,2000.0,22921.212121,0,0,0


In [9]:
# 평균가격 구했을 때 저장할 객체
pcs_price = pd.DataFrame(columns = sales_df.columns)
pcs_price.loc['Mean_Price'] = 'N'
pcs_price

,0,1,2,3,4,5,6,7,8,9,...,15880,15881,15882,15883,15884,15885,15886,15887,15888,15889
Mean_Price,N,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N


In [10]:
# 평균 가격 구하는 식
for i in range(15890):
    p = pd.DataFrame(price.iloc[:,i].value_counts().index)
    pcs_price.iloc[0,i] = p.sum()/(len(p)-1)

In [11]:
# 결측값 있는지 확인해보기
pcs_price.isnull().sum(1)

Mean_Price    224
dtype: int64

In [12]:
# 결측값 보면 대부분 전체 판매량이 0인 것들 -> 0으로 결측값 채우기
pcs_price = pcs_price.fillna(0)
pcs_price

,0,1,2,3,4,5,6,7,8,9,...,15880,15881,15882,15883,15884,15885,15886,15887,15888,15889
Mean_Price,7325.0,26333.75,10853.492063,4791.666667,4921.780492,6205.824561,7283.673864,11994.009813,5241.666667,15355.218626,...,0.0,0.0,0.0,0.0,8700.0,1888.169643,22178.258989,11712.896203,13600.0,49800.0


## 2. 제품 별 평균 가격을 통해 02-23 ~ 03-28 간 판매량과 곱하여 판매금액 산출

In [13]:
ntrain = train.loc['2023-02-23':'2023-03-28',:]
# ntrain

In [14]:
int_sales = pd.DataFrame(index = ntrain.index, columns = ntrain.columns)
int_sales

,0,1,2,3,4,5,6,7,8,9,...,15880,15881,15882,15883,15884,15885,15886,15887,15888,15889
2023-02-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
for j in range(15890):
    for i in range(34):
        int_sales.iloc[i,j] = float(ntrain.iloc[i,j]) * float(pcs_price.iloc[0,j])

int_sales

,0,1,2,3,4,5,6,7,8,9,...,15880,15881,15882,15883,15884,15885,15886,15887,15888,15889
2023-02-23,3598.45116,-1023.535323,20909.169138,11363.824379,57595.600759,14047.573933,0.0,462614.889416,371.356301,109393.571208,...,0.0,0.0,0.0,0.0,3024.261218,86208.119939,62208.657047,14980.10228,40668.53764,0.0
2023-02-24,3955.344278,-2204.881753,19825.977432,10832.342377,43287.652191,18551.666246,0.0,412868.550986,96.208968,112667.003517,...,0.0,0.0,0.0,0.0,3459.123437,110603.299783,-1181.712486,13809.777545,35232.76709,0.0
2023-02-25,3947.856663,-2192.302357,17113.954819,10299.8661,29224.963387,13916.982005,0.0,432635.085875,-444.340156,130553.20709,...,0.0,0.0,0.0,0.0,3458.126991,72274.024995,-40121.500003,11470.998999,27763.28869,0.0
2023-02-26,8310.818563,-2179.809547,15487.790115,9047.947834,33877.965692,14641.652578,0.0,449406.754775,-985.090733,95378.751873,...,0.0,0.0,0.0,0.0,3021.280598,40378.261765,-50124.996889,12637.611683,26394.196804,0.0
2023-02-27,14858.735661,220.622288,16576.517134,8514.241478,38283.554787,13791.669465,0.0,445204.554966,-995.896502,107120.464781,...,0.0,0.0,0.0,0.0,3020.246664,20143.227902,-2211.244146,10883.021612,28414.056632,0.0
2023-02-28,12759.37678,5173.811718,18838.051917,9595.361552,48195.114785,12949.283119,0.0,463961.979716,-908.763344,125550.344377,...,0.0,0.0,0.0,0.0,2495.28267,33960.377435,83212.798406,14374.647624,35683.660103,0.0
2023-03-01,16185.254507,5190.120368,17696.927266,9793.611725,64069.372298,8388.926381,0.0,452602.640009,-362.834818,70729.598673,...,0.0,0.0,0.0,0.0,2954.246107,56245.207054,68173.55405,14987.353067,39902.489991,0.0
2023-03-02,4174.373688,322.032611,20025.286304,10405.768742,56149.172798,14040.79222,0.0,434553.950654,441.616025,115357.816497,...,0.0,0.0,0.0,0.0,2794.479541,84710.672941,76592.175848,14204.785997,39326.695952,0.0
2023-03-03,4531.266807,-859.313792,18942.094609,9874.28674,41841.224215,18544.884533,0.0,384807.612224,166.468697,118631.248806,...,0.0,0.0,0.0,0.0,3229.341751,109105.852785,13201.806271,13034.461262,33890.925402,0.0
2023-03-04,4523.779192,-846.734397,16230.071996,9341.810463,27778.535406,13910.200292,0.0,404574.147113,-374.080432,136517.452395,...,0.0,0.0,0.0,0.0,3228.345314,70776.577997,-25737.981335,10695.682716,26421.447002,0.0


## 3. 기존 sales 데이터에 int_sales 데이터 합성

In [17]:
newsales = pd.concat([sales_df, int_sales,sales.loc['2023-03-29':,:]])
newsales

,0,1,2,3,4,5,6,7,8,9,...,15880,15881,15882,15883,15884,15885,15886,15887,15888,15889
2022-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-04,0.0,0.0,0.0,0.0,0.0,0.0,49000.0,0.0,63000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-05,0.0,0.0,0.0,0.0,0.0,0.0,147950.0,0.0,67200.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-31,0.0,44800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,28000.0,0.0,0.0,0.0,0.0,0.0,40460.0,0.0,0.0,0.0
2023-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,80920.0,0.0,0.0,0.0
2023-04-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,20230.0,0.0,0.0,0.0
2023-04-03,0.0,44800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,44500.0,24900.0,13000.0,0.0,0.0,20230.0,0.0,0.0,0.0


In [19]:
newsales.tail(n=20)

,0,1,2,3,4,5,6,7,8,9,...,15880,15881,15882,15883,15884,15885,15886,15887,15888,15889
2023-03-16,5326.218775,3013.168532,18257.520713,8489.657454,53256.316926,14027.228578,0.0,378430.776097,582.135501,127286.307167,...,0.0,0.0,0.0,0.0,2334.916153,81715.778643,105359.220326,12654.153713,36632.248462,0.0
2023-03-17,5683.111901,1831.822129,17174.329007,7958.175452,38948.368348,18531.320847,0.0,328684.275508,306.988173,130559.739476,...,0.0,0.0,0.0,0.0,2769.778362,106110.95843,41968.851991,11483.829012,31195.183138,0.0
2023-03-18,5675.6243,1844.401524,14462.306394,7425.699174,24885.679548,13896.636544,0.0,348450.648238,-233.560956,148445.94308,...,0.0,0.0,0.0,0.0,2768.781916,67781.683567,3029.065561,9145.050513,23724.409962,0.0
2023-03-19,10038.586208,1856.894334,12836.141701,6173.780908,29538.681859,14621.307062,0.0,365222.154979,-774.311528,113271.487879,...,0.0,0.0,0.0,0.0,2331.935515,35885.920262,-6974.430216,10311.663244,22354.023289,0.0
2023-03-20,16586.503321,4257.326196,13924.86872,5640.074557,33944.270959,13771.323887,0.0,361019.793011,-785.117292,125013.200818,...,0.0,0.0,0.0,0.0,2330.901581,15650.886329,40939.323636,8557.073208,24372.588329,0.0
2023-03-21,14487.144454,9210.515626,16186.403513,6721.194636,43855.830957,12928.937478,0.0,379777.055602,-697.984128,143443.080413,...,0.0,0.0,0.0,0.0,1805.937587,29468.0358,126363.367363,12048.699268,31640.897012,0.0
2023-03-22,17913.022188,9226.824276,15045.278852,6919.444813,59730.088415,8368.580672,0.0,368417.553736,-152.055603,88622.33474,...,0.0,0.0,0.0,0.0,2264.901024,51752.865343,111324.124182,12661.404745,35858.432126,0.0
2023-03-23,5902.141392,4358.736546,17373.637901,7531.601831,51809.888768,14020.446443,0.0,350368.702221,652.395246,133250.552595,...,0.0,0.0,0.0,0.0,2105.13445,80218.331154,119742.747156,11878.837711,35281.343298,0.0
2023-03-24,6259.034524,3177.390142,16290.446195,7000.119834,37501.940136,18524.538687,0.0,300622.201632,377.247918,136523.98495,...,0.0,0.0,0.0,0.0,2539.996668,104613.510941,56352.378733,10708.513022,29844.277961,0.0
2023-03-25,6251.546924,3189.969538,13578.423582,6467.643561,23439.251243,13889.854379,0.0,320388.574363,-163.301205,154410.188584,...,0.0,0.0,0.0,0.0,2539.000222,66284.236078,17412.592302,8369.734511,22373.504772,0.0


In [18]:
newsales.to_csv('newsales.csv')